# Visualise your labelled data

In [ ]:
import os
import io
import json
import boto3
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

from src.sm_utils import parse_s3_url

In [ ]:
with open("stack_outputs.json") as f:
    sagemaker_config = json.load(f)
s3_bucket = sagemaker_config["S3Bucket"]

## Get the labour of your hard work!
This notebook will download your labels from S3 and display several entries so you could check. 
Note that you must wait until your labelling job has finished.

In [ ]:
label_job_id = "{}-labelling-job".format(sagemaker_config["SolutionPrefix"])

In [ ]:
s3_client = boto3.client('s3')

# output manifest of the labelling job
output_manifest_location = "custom_data/output/{}/manifests/output/output.manifest".format(
    label_job_id)

item_object = s3_client.get_object(Bucket=s3_bucket, 
                                   Key=output_manifest_location)
body = item_object['Body'].read().decode('utf-8')

### Select the index you want to visualise

In [ ]:
index = 0

In [ ]:
entry = json.loads(body.split("\n")[index])

image_s3_key = parse_s3_url(entry["source-ref"])[1]
image_stream = io.BytesIO()

s3_client.download_fileobj(Bucket=s3_bucket, Key=image_s3_key, Fileobj=image_stream)

image_stream.seek(0)
img = mpimg.imread(image_stream, "png")

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 15))

for text_entry in entry["annotations"]["texts"]:
    text = text_entry["text"]
    bb = text_entry["bb"]
    x, y = [a['x'] for a in bb], [a['y'] for a in bb]
    xy = np.array([x, y])
    rect = patches.Polygon(xy.T, fill=False, color="r")
    ax.add_patch(rect)

    # Get the top left coord
    left_coords = np.argsort(xy[0, :])[:2]
    top_left = np.argmin(xy[1, left_coords])
    top_left_coord = xy[:, left_coords][:, top_left]
    ax.text(top_left_coord[0], top_left_coord[1], text, bbox=dict(facecolor='red', alpha=0.5))

ax.imshow(img, cmap='Greys_r')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

# Navigation
- Click [here](./2_label_own_data.ipynb) to start another labelling job
- Click [here](./4_model_training.ipynb) to train a model with your data